In [1]:
import pandas as pd

In [2]:
file = 'travistorrent_11_1_2017.csv'
chunksize = 1000000

In [3]:
for dataset in pd.read_csv(file, engine='c', chunksize=chunksize):
    process(dataset)

/home/wagner/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,6,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


NameError: name 'process' is not defined

In [4]:
#dataset = pd.read_csv('travistorrent_11_1_2017.csv',sep=',', engine='c', nrows=3000000, na_values=str, keep_default_na = bool, encoding='utf-8')
# nrows=1000000
'''
Signature:
    sep - separa por virgula
    nrows - limite de linhas lidas
    encoding - tipo de caracter
    engine - Mecanismo do analisador
    na_values - procura os padrões na|nan como str
    keep_default_na - define como nan os padroes econtrados em na_values
    
'''



'\nSignature:\n    sep - separa por virgula\n    nrows - limite de linhas lidas\n    encoding - tipo de caracter\n    engine - Mecanismo do analisador\n    na_values - procura os padrões na|nan como str\n    keep_default_na - define como nan os padroes econtrados em na_values\n    \n'

<font size=4/>
   * Elimia as colunas não utilizadas

In [4]:
# Eliminando colunas
dataset.drop([  'gh_diff_files_added',
                'gh_commits_in_push',
                'gh_diff_files_deleted',
                'gh_diff_files_modified',
                'gh_diff_tests_added',
                'gh_diff_tests_deleted',
                'gh_diff_src_files',
                'gh_diff_doc_files',
                'gh_diff_other_files',
                'gh_sloc',
                'gh_test_lines_per_kloc',
                'gh_test_cases_per_kloc',
            
                'tr_duration',
                'tr_log_lan',
                'tr_log_num_tests_run',
                'tr_log_setup_time',
                'tr_log_analyzer',
                'tr_log_frameworks',
                'tr_log_bool_tests_ran',
                'tr_log_bool_tests_failed',
                'tr_log_tests_failed',
                'tr_log_testduration',
                'tr_log_buildduration'], axis=1, inplace=True)

In [6]:
def categorizaPush(valor):
    if(valor < 20):
        return 0

<font size=4/> 
   * Filtra o projeto

In [5]:
df_ruby = dataset.loc[dataset['gh_project_name']=='rails/rails'] # Filtro penas o projeto rails/rails

In [18]:
ruby_master = df_ruby.loc[df_ruby['git_branch']== 'master']

In [30]:
teste = ruby_master.loc[ruby_master['gh_first_commit_created_at'] != 'NaN'] # teste

In [34]:
#df_statusFail = df_java.loc[df_java['tr_status']=='failed'] # Filtra apenas as bulds quebrada


False

Onde: num_commitsTPP significa o número total de commits por projeto, obtido após agrupamento da amostra por gh_project_name e somando git_num_commits; 


tempoDP significa o tempo total de duração de cada projeto em dias, obtido após agrupamento da amostra por gh_project_name 

e 


diminuindo a data máxima da data mínima extraída de gh_first_commit_created_at no grupo;


MediaPartPP significa a média de participantes por projeto, obtido após agrupamento da amostra por gh_project_name,



somando gh_team_size 


e dividindo pelo número de builds em cada grupo, 

tendo em vista que gh_team_size tem pequenas variações entre builds ao longo do projeto.

Para cada linguagem obtemos a TxCommitsPP e realizamos o teste de normalidade de Shapiro-Wilk e plotamos o gráfico quantil-quantil (QQ). Como as duas distribuições se desviaram da distribuição normal de forma significante, observamos no gráfico QQ que as duas amostras possuem forma de distribuição semelhante confirmada pelo teste de variância. Assim, para verificar se existe diferença estatística significante entre as duas distribuições, optamos por usar o teste não paramétrico U de Mann-Whitney. Os gráficos QQ podem ser vistos nas Figuras 1 e 2, abaixo.